In [1]:
#!pip install gradio

In [2]:
import os 
import requests 
import openai 
import anthropic 
import google.generativeai 
from bs4 import BeautifulSoup 
from dotenv import load_dotenv 
from typing import List

import gradio as gr

In [3]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [4]:
openai = openai.OpenAI() 
claude = anthropic.Anthropic() 
google.generativeai.configure()

In [5]:
system_message = "You are helpful assistant"

In [6]:
def gpt_message(prompt):
    messages = [
        {'role': 'system', 'content': system_message}, 
        {'role': 'user', 'content': prompt}
    ]
    result = openai.chat.completions.create(model='gpt-4o-mini', messages=messages) 
    return completions.choics[0].message.content

In [7]:
def shout(text):
    return text.upper()

In [8]:
# gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()
## Created dataset file at: .gradio\flagged\dataset1.csv
## to remove 'flagged' folder inside 'gradio' folder 
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode='never').launch() 
## to create a public access
# gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


#### Text box with lines 
    inputs=[gr.Textbox(label="Your message", lines=6)],            
    outputs=[gr.Textbox(label="Response:", lines=8)],       

#### Response in Markdown

In [9]:
system_message = "You are a helpful assistant that responds in markdown"

### Streaming

In [10]:
def gpt_stream(prompt): 
    messages = [{'role': 'system', 'content': system_message},{'role':'user', 'content': prompt}] 
    stream = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, stream=True) 

    result=""
    for chunk in stream:
        result += chunk.choices[0].delta.content or "" 
        yield result

In [11]:
def claude_stream(prompt):
    stream_claude = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=500,
        temperature=0.7,
        system=system_message,
        messages=[{'role': 'user', 'content': prompt}]
    )
    response =""
    with stream_claude as stream: 
        for text in stream.text_stream: 
            response += text or "" 
            yield response

In [12]:
def gemini_stream(prompt):
    #google.generativeai.configure(api_key="GOOGLE_API_KEY")
    model = google.generativeai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt, stream=True)

    results=""
    for chunk in response:
        results += chunk.text or ""
        #print(chunk.text, end="")
        yield results
    

In [13]:
def stream_model(prompt, model): 
    if model=="GPT": 
        result = gpt_stream(prompt) 
    elif model=="Claude": 
        result = claude_stream(prompt) 
    elif model=="Gemini":
        result = gemini_stream(prompt)
    else: 
        raise ValueError("Unknown model") 

    for chunk in result: 
        yield chunk 

In [14]:
view = gr.Interface(
    fn=stream_model, 
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(['GPT', 'Claude', 'Gemini'], label="Select Model") ],
    outputs=[gr.Markdown(label="Response:")], 
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Building a company brochure generator 

In [15]:
class Website: 
    url: str 
    title: str
    text: str

    def __init__(self, url): 
        self.url = url 
        request = requests.get(url) 
        self.body = request.content 
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "Title Not Found"
        for irrelevant in soup.body(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self): 
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [16]:
system_prompt = "You are an assistant tha analyzes the contents of a company website landing page and creates a short brochure about a company for prospective customers, investors and recruits. Respond in markdown"

In [17]:
def stream_brochure(company_name, url, model): 
    prompt=f"Please generate a company brochure for {company_name}. Here is their landing page:\n" 
    prompt += Website(url).get_contents() 
    if model=="GPT": 
        result=gpt_stream(prompt)  
    elif model=="Claude": 
        result=claude_stream(prompt) 
    elif model=="Gemini":
        result=gemini_stream(prompt)
    else: 
        raise ValueError("Unknow model") 

    for chunk in result: 
        yield chunk 


In [18]:
view = gr.Interface(
    fn=stream_brochure, 
    inputs=[
        gr.Textbox(label="Company name: "),  
        gr.Textbox(label="Landing page URL:"), 
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model") 
    ], 
    outputs=[gr.Markdown(label="Brochure: ")], 
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
